In [37]:
import math
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn.preprocessing import *
from sklearn import ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt  
from sklearn.metrics import log_loss
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score



In [ ]:
import math
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import csv
from datetime import *
from sklearn.preprocessing import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold



In [38]:

train_df = pd.read_csv('../temp/fusai_train_df_all.csv')
test_df = pd.read_csv('../temp/fusai_test_df_all.csv')


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (59,87,109,110,125) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (84,122) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
class XgbModel:
    def __init__(self, feaNames=None, params={}):
        self.feaNames = feaNames
        self.params = {
            'objective': 'binary:logistic',
            'eval_metric':'auc',
            'silent': True,
            'eta': 0.05,
            'max_depth': 6,
            'gamma': 1,
            'subsample': 0.9,
            'colsample_bytree': 0.95,
            'min_child_weight': 3,
            'max_delta_step': 1,
            'lambda': 30,
#             'nthread': 20,
        }
        for k,v in params.items():
            self.params[k] = v
        self.clf = None

    def train(self, X, y, train_size=1, test_size=0.1, verbose=True, num_boost_round=1000, early_stopping_rounds=3):
        X = X.astype(float)
        if train_size==1:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            X_train, y_train = X, y
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size)
        dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=self.feaNames)
        dval = xgb.DMatrix(X_test, label=y_test, feature_names=self.feaNames)
        watchlist = [(dtrain,'train'),(dval,'val')]
        clf = xgb.train(
            self.params, dtrain, 
            num_boost_round = num_boost_round, 
            evals = watchlist, 
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        self.clf = clf

    def trainCV(self, X, y, nFold=10, verbose=True, num_boost_round=8000, early_stopping_rounds=100):
        X = X.astype(float)
        dtrain = xgb.DMatrix(X, label=y, feature_names=self.feaNames)
        cvResult = xgb.cv(
            self.params, dtrain, 
            num_boost_round = num_boost_round, 
            nfold = nFold,
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        clf = xgb.train(
            self.params, dtrain, 
            num_boost_round = cvResult.shape[0], 
        )
        self.clf = clf

    def gridSearch(self, X, y, nFold=3, verbose=1, num_boost_round=130):
        paramsGrids = {
            # 'n_estimators': [50+5*i for i in range(0,30)],
            'gamma': [0,0.01,0.05,0.1,0.5,1,5,10,50,100],
            # 'max_depth': list(range(3,10)),
            'min_child_weight': list(range(0,10)),
            'subsample': [1-0.05*i for i in range(0,8)],
            'colsample_bytree': [1-0.05*i for i in range(0,10)],
            # 'reg_alpha': [0+2*i for i in range(0,10)],
            'reg_lambda': [0+50*i for i in range(0,10)],            
            'max_delta_step': [0+1*i for i in range(0,8)],
        }
        for k,v in paramsGrids.items():
            gsearch = GridSearchCV(
                estimator = xgb.XGBClassifier(
                    max_depth = self.params['max_depth'], 
                    gamma = self.params['gamma'],
                    learning_rate = self.params['eta'],
                    max_delta_step = self.params['max_delta_step'],
                    min_child_weight = self.params['min_child_weight'],
                    subsample = self.params['subsample'],
                    colsample_bytree = self.params['colsample_bytree'],
                    silent = self.params['silent'],
                    reg_lambda = self.params['lambda'],
                    n_estimators = num_boost_round
                ),
                # param_grid = paramsGrids,
                param_grid = {k:v},
                scoring = 'neg_log_loss',
                cv = nFold,
                verbose = verbose,
                n_jobs = 4
            )
            gsearch.fit(X, y)
            print(pd.DataFrame(gsearch.cv_results_))
            print(gsearch.best_params_)
        exit()

    def predict(self, X):
        X = X.astype(float)
        return self.clf.predict(xgb.DMatrix(X, feature_names=self.feaNames))

    def getFeaScore(self, show=False):
        fscore = self.clf.get_score()
        feaNames = fscore.keys()
        scoreDf = pd.DataFrame(index=feaNames, columns=['importance'])
        for k,v in fscore.items():
            scoreDf.loc[k, 'importance'] = v
        if show:
            print(scoreDf.sort_index(by=['importance'], ascending=False))
        return scoreDf

# 划分训练集和测试集
def trainTestSplit(df, splitDate=pd.to_datetime('2018-09-23'), trainPeriod=3, testPeriod=1):
    trainDf = df[(df.context_timestamp<splitDate)&(df.context_timestamp>=splitDate-timedelta(days=trainPeriod))]
    testDf = df[(df.context_timestamp>=splitDate)&(df.context_timestamp<splitDate+timedelta(days=testPeriod))]
    return (trainDf, testDf)


# 统计预测误差
def countDeltaY(predictSeries, labelSeries, show=True, title='', subplot=None):
    deltaSeries = predictSeries - labelSeries
    if subplot!=None:
        plt.subplot(subplot[0], subplot[1], subplot[2])
    deltaSeries.plot(style='b-')
    plt.title(title)
    if show:
        plt.show()
    return deltaSeries

# 获取stacking下一层数据集
def getOof(clf, trainX, trainY, testX, nFold=5, stratify=False):
    oofTrain = np.zeros(trainX.shape[0])
    oofTest = np.zeros(testX.shape[0])
    oofTestSkf = np.zeros((testX.shape[0], nFold))
    if stratify:
        kf = StratifiedKFold(n_splits=nFold, shuffle=True)
    else:
        kf = KFold(n_splits=nFold, shuffle=True)
    for i, (trainIdx, testIdx) in enumerate(kf.split(trainX, trainY)):
        kfTrainX = trainX[trainIdx]
        kfTrainY = trainY[trainIdx]
        kfTestX = trainX[testIdx]
        clf.trainCV(kfTrainX, kfTrainY, verbose=False)
        oofTrain[testIdx] = clf.predict(kfTestX)
        oofTestSkf[:,i] = clf.predict(testX)
    oofTest[:] = oofTestSkf.mean(axis=1)
    return oofTrain, oofTest





In [40]:
chusai_QY_xinxi_train_df = pd.read_csv('../../chusai/data/train/企业基本信息&高管信息&投资信息.csv')
fusai_QY_xinxi_train_df = pd.read_csv('../data/train/企业基本信息&高管信息&投资信息.csv')
QY_xinxi_train_df = pd.concat([chusai_QY_xinxi_train_df, fusai_QY_xinxi_train_df])
QY_xinxi_train_df.rename(columns={'企业名称':'qymc', '注册号':'zch', '统一社会信用代码':'tyshxydm', '注册资金':'zczj', '注册资本(金)币种名称':'zczbbzmc', 
                                  '企业(机构)类型名称':'qyjglxmc', '行业门类代码':'hymldm', '成立日期':'clrq', '核准日期':'hzrq', '住所所在地省份':'zsszdsf', 
                                  '姓名':'xm', '法定代表人标志':'fddbrbz', '首席代表标志':'sxdbbz', '职务':'zw', '投资人':'tzr', '出资比例':'czbl'}, inplace=True)
after_train_df = QY_xinxi_train_df[['qymc', 'zch', 'tyshxydm', 'zczj', 'zczbbzmc', 'qyjglxmc', 'hymldm', 'clrq', 'hzrq', 'zsszdsf']]
after_train_df.drop_duplicates(['qymc'], keep='first', inplace=True)
train_suzhou_set = set(after_train_df[after_train_df.zsszdsf == '江苏省']['qymc'])
train_df['is_suzhou'] = train_df['qymc'].map(lambda x : 1 if x in train_suzhou_set else 0)

QY_xinxi_test_df = pd.read_csv('../data/test/企业基本信息&高管信息&投资信息.csv')
QY_xinxi_test_df.rename(columns={'企业名称':'qymc', '注册号':'zch', '统一社会信用代码':'tyshxydm', '注册资金':'zczj', '注册资本(金)币种名称':'zczbbzmc', 
                                  '企业(机构)类型名称':'qyjglxmc', '行业门类代码':'hymldm', '成立日期':'clrq', '核准日期':'hzrq', '住所所在地省份':'zsszdsf',
                                  '姓名':'xm', '法定代表人标志':'fddbrbz', '首席代表标志':'sxdbbz', '职务':'zw', '投资人':'tzr', '出资比例':'czbl'}, inplace=True)
after_test_df = QY_xinxi_test_df[['qymc', 'zch', 'tyshxydm', 'zczj', 'zczbbzmc', 'qyjglxmc', 'hymldm', 'clrq', 'hzrq', 'zsszdsf']]
after_test_df.drop_duplicates(['qymc'], keep='first', inplace=True)
print(len(after_test_df))
suzhou_set = set(after_test_df[after_test_df.zsszdsf == '江苏省']['qymc'])
print(len(suzhou_set))
test_df['is_suzhou'] = test_df['qymc'].map(lambda x : 1 if x in suzhou_set else 0)
print(len(test_df[test_df.is_suzhou == 1]))
print(len(test_df))

categorical_feature_shixin = ['zsszdsf_encoder', 'qyjglxmc_encoder', 'hymldm_encoder', 
                       'zczbbzmc_encoder', 'fzjglx', 'fzhydm', 'fzhyml_encoder', 'fzjgzt',
                      'fzjgsf', 'qylxmc_encoder', 'xzgh', 'sshydm_encoder', 'zl_encoder', 
                      'SGSFRXZXKXX_xknr_encoder', 'QY_fddbrbz_gs', 'SGSFRXZXKXX_xmmc_encoder']

categorical_feature_chufa = ['zsszdsf_encoder', 'qyjglxmc_encoder', 'hymldm_encoder', 
                       'zczbbzmc_encoder', 'fzjglx', 'fzhydm', 'fzhyml_encoder', 'fzjgzt',
                      'fzjgsf', 'qylxmc_encoder', 'xzgh', 'sshydm_encoder', 'zl_encoder']




/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


17534
14802
14802
17534


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
FRXZXKZX_test_df = pd.read_csv('../data/test/法人行政许可注（撤、吊）销信息.csv')
fr_qymc_set = set(FRXZXKZX_test_df['企业名称'])

test_df['is_fr'] = test_df['qymc'].map(lambda x : 1 if x in fr_qymc_set else 0)

QYFZCHRD_test_df = pd.read_csv('../data/test/企业非正常户认定.csv')
fzc_qymc_set = set(QYFZCHRD_test_df['企业名称'])

test_df['is_fzc'] = test_df['qymc'].map(lambda x : 1 if x in fzc_qymc_set else 0)



In [42]:
FZJGXX_test_df = pd.read_csv('../data/test/分支机构信息.csv')
fz_qymc_set = set(FZJGXX_test_df['企业名称'])

test_df['is_fz'] = test_df['qymc'].map(lambda x : 1 if x in fz_qymc_set else 0)


In [43]:
SDBDJQQJMCF_test_df = pd.read_csv('../data/test/双打办打击侵权假冒处罚案件信息.csv')
qq_qymc_set = set(SDBDJQQJMCF_test_df['企业名称'])

test_df['is_qq'] = test_df['qymc'].map(lambda x : 1 if x in qq_qymc_set else 0)



In [44]:
ZPSJ_test_df = pd.read_csv('../../check/data/test/招聘数据.csv')
zp_qymc_set = set(ZPSJ_test_df['企业名称'])

test_df['is_zp'] = test_df['qymc'].map(lambda x : 1 if x in zp_qymc_set else 0)


In [45]:
lake_rule1_test_df = pd.read_csv('../../check/data/test/企业基本信息&高管信息&投资信息.csv')
temp = lake_rule1_test_df[['企业名称','统一社会信用代码']].groupby(['企业名称'], as_index=False)['统一社会信用代码'].agg({'is_tyshxydm_many':'nunique'})
temp.rename(columns={'企业名称':'qymc'}, inplace=True)
test_df = pd.merge(test_df,temp,on=['qymc'],how='left')


In [46]:
lake_rule2_test_df = pd.read_csv('../../check/data/test/企业基本信息&高管信息&投资信息.csv')
temp = lake_rule2_test_df[['企业名称','注册号']].groupby(['企业名称'], as_index=False)['注册号'].agg({'is_zch_many':'nunique'})
temp.rename(columns={'企业名称':'qymc'}, inplace=True)
test_df = pd.merge(test_df,temp,on=['qymc'],how='left')


In [47]:
shixin_fea = ['zczj', 'fzjgsf', 'fzjgzt', 'fzhydm', 'fzjgqx', 'fzjglx', 'sjly', 'rwbh', 
#               'zckbzj', 
       'QYFZCHRD_yynx', 'sshydm_encoder', 'xzgh', 'qyjd', 'qywd', 
       'QYBZRYXX_number', 'QYFZCHRD_number', 'QYSWDJXX_number', 'FRXZXKZX_number',
       'SDBDJQQJMCF_number', 'SGSFRXZXKXX_number', 'XKZZNJXX_number',
       'ZPSJ_number', 'ZZDJBGXX_number', 'QYJL_number', 
      'zczbbzmc_encoder', 'hymldm_encoder', 'zsszdsf_encoder', 'fzhyml_encoder',
       'xxtgbmmc_encoder', 'zl_encoder', 'sszj_encoder', 'fzjgmc_encoder',
       'qyjglxmc_encoder', 'qylxmc_encoder', 'QYFZCHRD_swglm_encoder',
       'QYFZCHRD_gljg_encoder', 'QYFZCHRD_djzclx_encoder', 
        'SGSFRXZXKXX_dfbm', 'SGSFRXZXKXX_xknr_encoder', 'SGSFRXZXKXX_xmmc_encoder', 'SGSFRXZXKXX_splb_encoder', 
       'SGSFRXZXKXX_xkjg_encoder', 'SGSFRXZXKXX_xysyfw_encoder', 'SGSFRXZXKXX_sjzt2_encoder', 
        'XKZZNJXX_njnd_encoder', 'XKZZNJXX_njjgmc_encoder', 'XKZZNJXX_njsxmc_encoder', 
         'ZZDJBGXX_zzmc_encoder', 'ZZDJBGXX_zyfw_encoder', 'ZZDJBGXX_rdjgqc_encoder', 'ZZDJBGXX_zzzsbh_encoder', 
      'QY_tzrgs', 'QY_zwzls', 'QY_fddbrbz_gs', 'QY_sxdbbz_gs', 
              
        'fzsw_deltaDay', 'clhz_deltaDay', 
       'clrq_month', 'clrq_day', 'hzrq_year', 'hzrq_month', 'hzrq_day', 'clrq_year',
              
       'ZPSJ_zprs_max', 'ZPSJ_zprs_min', 'ZPSJ_zprs_mean', 'ZPSJ_zprs_std',
              
#               'jgdzzs_top3_encoder', 'jgdzzs_top6_encoder',
               'QYSWDJXX_fddbrzjmc_encoder', 'QYSWDJXX_shdw_encoder',
 'QYSWDJXX_qy_encoder', 'QYSWDJXX_shjg_encoder', 'QYSWDJXX_djzclx_encoder',
              
#       'ZPSJ_zylb_kinds', 'ZPSJ_gzjy_kinds',
#        'ZPSJ_gzdd_kinds', 'ZPSJ_zprs_kinds', 'ZPSJ_zwyx_kinds',
#        'ZPSJ_zdxl_kinds', 'ZPSJ_zprq_kinds',
              
#         'SGSFRXZXKXX_xkjdjz_deltaDay',
#       'SGSFRXZXKXX_xkjzq_year', 'SGSFRXZXKXX_xkjzq_month', 'SGSFRXZXKXX_xkjzq_day', 
#       'SGSFRXZXKXX_xkjdrq_year', 'SGSFRXZXKXX_xkjdrq_month', 'SGSFRXZXKXX_xkjdrq_day',  
    
       
#        'clxkjz_deltaDay', 'hzxkjz_deltaDay', 'clxkjd_deltaDay', 'hzxkjd_deltaDay', 
#         'njhz_deltaDay', 'njcl_deltaDay', 
#        'XKZZNJXX_njrq_year', 'XKZZNJXX_njrq_month', 'XKZZNJXX_njrq_day', 
#        'ZZDJBGXX_zzsxq_year', 'ZZDJBGXX_zzsxq_month', 'ZZDJBGXX_zzsxq_day', 
#        'ZZDJBGXX_zzjzq_year', 'ZZDJBGXX_zzjzq_month', 'ZZDJBGXX_zzjzq_day', 
       'ZZDJBGXX_jzsx_deltaDay', 'zzsxcl_deltaDay', 'zzjzcl_deltaDay', 'zzsxhz_deltaDay', 'zzjzhz_deltaDay', 
       
#        'czbl_max', 'czbl_min', 'czbl_mean', 'czbl_std',
       
       'jgdzzs_encoder', 'jyfw_encoder', 'sshymc_encoder',
#        'fzrq_encoder', 'bghzrq_encoder',
#      'gszch_encoder', 'fddbrzjhm_encoder', 'zzjgdm_encoder',
       
#        'ZPSJ_wzmc_max', 'ZPSJ_wzmc_min', 'ZPSJ_wzmc_mean',
#  'ZPSJ_wzmc_std', 'ZPSJ_gzjy_max', 'ZPSJ_gzjy_min', 'ZPSJ_gzjy_mean',
#  'ZPSJ_gzjy_std', 'ZPSJ_gzdd_max', 'ZPSJ_gzdd_min', 'ZPSJ_gzdd_mean',
#  'ZPSJ_gzdd_std', 'ZPSJ_zylb_max', 'ZPSJ_zylb_min', 'ZPSJ_zylb_mean',
#  'ZPSJ_zylb_std', 'ZPSJ_zwyx_max', 'ZPSJ_zwyx_min', 'ZPSJ_zwyx_mean',
#  'ZPSJ_zwyx_std', 'ZPSJ_zdxl_max', 'ZPSJ_zdxl_min', 'ZPSJ_zdxl_mean',
#  'ZPSJ_zdxl_std', 'ZPSJ_ywzj_max', 'ZPSJ_ywzj_min', 'ZPSJ_ywzj_mean',
#  'ZPSJ_ywzj_std',
       
#        'SGSFRXZXKXX_is_shixin_sjly_rate', 'SGSFRXZXKXX_is_chufa_sjly_rate',
#  'SGSFRXZXKXX_is_shixin_cjrID_rate', 'SGSFRXZXKXX_is_chufa_cjrID_rate',
#  'SGSFRXZXKXX_is_shixin_xxtgbmbm_rate',
#  'SGSFRXZXKXX_is_chufa_xxtgbmbm_rate', 'SGSFRXZXKXX_is_shixin_splb_rate',
#  'SGSFRXZXKXX_is_chufa_splb_rate', 'SGSFRXZXKXX_is_shixin_dfbm_rate',
#  'SGSFRXZXKXX_is_chufa_dfbm_rate', 'SGSFRXZXKXX_is_shixin_xkjg_rate',
#  'SGSFRXZXKXX_is_chufa_xkjg_rate',
       
      ]



In [48]:
temp_shixin_df = train_df[train_df.is_shixin == 1].copy()
temp_notshixin_df = train_df[(train_df.is_shixin == 0)].copy()
# temp_notshixin_df = temp_notshixin_df.sample(frac=0.9, random_state=2018, replace=True)
temp_train_df_list = list()
skf_dataset = StratifiedKFold(n_splits=4, random_state=2018, shuffle=True)
for index, (train_index, test_index) in enumerate(skf_dataset.split(temp_notshixin_df[shixin_fea], temp_notshixin_df['is_shixin'])):
    temp = temp_notshixin_df.iloc[train_index].copy()
    temp = pd.concat([temp, temp_shixin_df])
    temp.reset_index(inplace=True)
    temp_train_df_list.append(temp)



In [49]:
shixin_params = {
        'eta': 0.05,
        'max_depth': 6,
        'gamma': 1,
        'subsample': 0.9,
        'colsample_bytree': 0.95,
        'min_child_weight': 1,
        'max_delta_step': 1,
        'lambda': 30,
#             'nthread': 20,
    }


In [50]:
xgbModel_shixin = XgbModel(feaNames=shixin_fea, params=shixin_params)

for i in range(4):
    temp_train_df = temp_train_df_list[i]
    test_df['FORTARGET1_round' + str(i)] = 0
    xgbModel_shixin.trainCV(temp_train_df[shixin_fea].values, temp_train_df['is_shixin'].values)
    test_df.loc[:, 'FORTARGET1_round' + str(i)] = xgbModel_shixin.predict(test_df[shixin_fea].values)



[0]	train-auc:0.501157+0.000944835	test-auc:0.499976+7.17e-05
[1]	train-auc:0.50135+0.000884073	test-auc:0.499976+7.17e-05
[2]	train-auc:0.501348+0.00088225	test-auc:0.499976+7.17e-05
[3]	train-auc:0.501346+0.000881394	test-auc:0.499976+7.17e-05
[4]	train-auc:0.501346+0.000881394	test-auc:0.499976+7.17e-05
[5]	train-auc:0.501346+0.000881394	test-auc:0.499976+7.17e-05
[6]	train-auc:0.501345+0.000880579	test-auc:0.499964+7.64628e-05
[7]	train-auc:0.501469+0.000761694	test-auc:0.500631+0.00201345
[8]	train-auc:0.50172+0.000307585	test-auc:0.501839+0.00289489
[9]	train-auc:0.501719+0.000306671	test-auc:0.501827+0.00290265
[10]	train-auc:0.501717+0.000308626	test-auc:0.501827+0.00290265
[11]	train-auc:0.501712+0.000310923	test-auc:0.501827+0.00290265
[12]	train-auc:0.501711+0.000310073	test-auc:0.501815+0.00291133
[13]	train-auc:0.501708+0.000314077	test-auc:0.501815+0.00291133
[14]	train-auc:0.501706+0.000312253	test-auc:0.501815+0.00291133
[15]	train-auc:0.501703+0.000310552	test-auc:0.50

[130]	train-auc:0.935351+0.00150997	test-auc:0.916189+0.0138482
[131]	train-auc:0.935653+0.00148994	test-auc:0.916402+0.0138111
[132]	train-auc:0.93595+0.00146124	test-auc:0.916617+0.0137899
[133]	train-auc:0.936271+0.00147562	test-auc:0.916917+0.0138594
[134]	train-auc:0.936495+0.00148481	test-auc:0.917114+0.0138295
[135]	train-auc:0.936796+0.00150681	test-auc:0.917218+0.0138337
[136]	train-auc:0.93706+0.00152324	test-auc:0.917292+0.0137104
[137]	train-auc:0.937356+0.00151648	test-auc:0.917581+0.013613
[138]	train-auc:0.937645+0.00154992	test-auc:0.91784+0.0135555
[139]	train-auc:0.93788+0.00152043	test-auc:0.917987+0.0135466
[140]	train-auc:0.938184+0.00157456	test-auc:0.918231+0.0135367
[141]	train-auc:0.938492+0.00160907	test-auc:0.91841+0.0135014
[142]	train-auc:0.938744+0.0016229	test-auc:0.918509+0.01356
[143]	train-auc:0.938976+0.00165022	test-auc:0.918667+0.0134989
[144]	train-auc:0.939196+0.00160025	test-auc:0.91884+0.0134003
[145]	train-auc:0.939419+0.00160329	test-auc:0.918

[259]	train-auc:0.957226+0.00130089	test-auc:0.925849+0.0109807
[260]	train-auc:0.957343+0.0012729	test-auc:0.925886+0.0109442
[261]	train-auc:0.957454+0.00127199	test-auc:0.925902+0.0109794
[262]	train-auc:0.957573+0.00127259	test-auc:0.925923+0.0109317
[263]	train-auc:0.957656+0.00121886	test-auc:0.925995+0.0108758
[264]	train-auc:0.957776+0.00123886	test-auc:0.926006+0.0108634
[265]	train-auc:0.957861+0.00124893	test-auc:0.926018+0.0108812
[266]	train-auc:0.95798+0.00122995	test-auc:0.925998+0.0108406
[267]	train-auc:0.958071+0.00126809	test-auc:0.925998+0.0108595
[268]	train-auc:0.958184+0.00129701	test-auc:0.926003+0.0108485
[269]	train-auc:0.958295+0.00130784	test-auc:0.92607+0.0108098
[270]	train-auc:0.95839+0.00131812	test-auc:0.926142+0.0107861
[271]	train-auc:0.958506+0.00133214	test-auc:0.926174+0.0107621
[272]	train-auc:0.958604+0.00131665	test-auc:0.926212+0.0107443
[273]	train-auc:0.958722+0.00127533	test-auc:0.92623+0.0106667
[274]	train-auc:0.958854+0.00126889	test-auc:

[388]	train-auc:0.969364+0.00104301	test-auc:0.92659+0.0106475
[389]	train-auc:0.969456+0.00106745	test-auc:0.926563+0.0106055
[390]	train-auc:0.969521+0.00105121	test-auc:0.926574+0.0105849
[391]	train-auc:0.969591+0.00104351	test-auc:0.926603+0.0105432
[392]	train-auc:0.969667+0.00104117	test-auc:0.92657+0.0105512
[393]	train-auc:0.969742+0.00103634	test-auc:0.926531+0.0105214
[394]	train-auc:0.969807+0.00102409	test-auc:0.926551+0.0104929
[395]	train-auc:0.969886+0.00101634	test-auc:0.926543+0.0104677
[396]	train-auc:0.969969+0.00100414	test-auc:0.926577+0.0104635
[397]	train-auc:0.970043+0.000991104	test-auc:0.926587+0.0104479
[398]	train-auc:0.970124+0.000998146	test-auc:0.926585+0.0104515
[399]	train-auc:0.970201+0.000990755	test-auc:0.926609+0.0104389
[400]	train-auc:0.970295+0.000979064	test-auc:0.926585+0.0104603
[401]	train-auc:0.970377+0.00099058	test-auc:0.926589+0.0104016
[402]	train-auc:0.970437+0.00100295	test-auc:0.926599+0.0103654
[403]	train-auc:0.970519+0.00100387	te

[98]	train-auc:0.922343+0.00167869	test-auc:0.904408+0.0158219
[99]	train-auc:0.922775+0.00162549	test-auc:0.904831+0.0157729
[100]	train-auc:0.923297+0.00165564	test-auc:0.905126+0.0158286
[101]	train-auc:0.923672+0.00168133	test-auc:0.905248+0.015937
[102]	train-auc:0.924147+0.0016514	test-auc:0.905554+0.0159869
[103]	train-auc:0.924626+0.00166693	test-auc:0.906284+0.0156494
[104]	train-auc:0.925266+0.00155017	test-auc:0.907033+0.0156581
[105]	train-auc:0.925753+0.00142785	test-auc:0.907556+0.0156352
[106]	train-auc:0.926253+0.00139462	test-auc:0.908086+0.0153626
[107]	train-auc:0.926661+0.00141323	test-auc:0.908602+0.015126
[108]	train-auc:0.927109+0.00149898	test-auc:0.908943+0.0152412
[109]	train-auc:0.927537+0.00157462	test-auc:0.909419+0.0151025
[110]	train-auc:0.928021+0.00149464	test-auc:0.909752+0.0152314
[111]	train-auc:0.928391+0.00144992	test-auc:0.909919+0.0153349
[112]	train-auc:0.928903+0.00130685	test-auc:0.910438+0.0154499
[113]	train-auc:0.929285+0.00131729	test-auc:

...

[364]	train-auc:0.967391+0.000937338	test-auc:0.926678+0.0122948
[365]	train-auc:0.96748+0.000944604	test-auc:0.926666+0.0123281
[366]	train-auc:0.967549+0.000921801	test-auc:0.926678+0.0123063
[367]	train-auc:0.967647+0.000935554	test-auc:0.926685+0.0122878
[368]	train-auc:0.96771+0.000951681	test-auc:0.926704+0.012263
[369]	train-auc:0.96779+0.000956007	test-auc:0.926754+0.0122641
[370]	train-auc:0.967875+0.00093761	test-auc:0.926701+0.0122565
[371]	train-auc:0.967962+0.00091187	test-auc:0.926683+0.0122509
[372]	train-auc:0.96807+0.000903753	test-auc:0.926687+0.0122478
[373]	train-auc:0.968167+0.000911288	test-auc:0.926691+0.0122466
[374]	train-auc:0.96826+0.000915371	test-auc:0.926648+0.0122854
[375]	train-auc:0.968362+0.000916737	test-auc:0.926681+0.0122703
[376]	train-auc:0.96843+0.000925124	test-auc:0.92672+0.0122779
[377]	train-auc:0.968507+0.000935689	test-auc:0.926745+0.0122568
[378]	train-auc:0.968589+0.000954136	test-auc:0.926742+0.0122693
[379]	train-auc:0.968684+0.00093305

[491]	train-auc:0.976368+0.000751588	test-auc:0.926422+0.0119297
[492]	train-auc:0.976424+0.000757948	test-auc:0.926409+0.0119301
[493]	train-auc:0.976484+0.0007449	test-auc:0.926411+0.0119236
[494]	train-auc:0.976533+0.000758765	test-auc:0.926417+0.0119178
[495]	train-auc:0.976604+0.000766567	test-auc:0.926399+0.011931
[496]	train-auc:0.976667+0.000749071	test-auc:0.926391+0.0119242
[497]	train-auc:0.976728+0.000752555	test-auc:0.926402+0.0118981
[498]	train-auc:0.97678+0.000760221	test-auc:0.926337+0.0119156
[499]	train-auc:0.97685+0.00074949	test-auc:0.926346+0.0119319
[500]	train-auc:0.97691+0.000751171	test-auc:0.926325+0.0119536
[501]	train-auc:0.976971+0.000754476	test-auc:0.926291+0.0119287
[502]	train-auc:0.977025+0.000746264	test-auc:0.926297+0.0119437
[503]	train-auc:0.977073+0.000754984	test-auc:0.926291+0.0119643
[504]	train-auc:0.977137+0.000754449	test-auc:0.926294+0.01194
[505]	train-auc:0.977194+0.000762903	test-auc:0.926315+0.0119291
[506]	train-auc:0.977254+0.0007823

[114]	train-auc:0.929386+0.00158174	test-auc:0.9113+0.0150325
[115]	train-auc:0.929703+0.00166029	test-auc:0.911579+0.0149355
[116]	train-auc:0.930081+0.00167716	test-auc:0.911883+0.0147445
[117]	train-auc:0.930448+0.00164118	test-auc:0.912321+0.0144068
[118]	train-auc:0.930852+0.00149049	test-auc:0.912598+0.0145166
[119]	train-auc:0.931231+0.00141784	test-auc:0.912957+0.0144297
[120]	train-auc:0.93159+0.00138068	test-auc:0.913182+0.014352
[121]	train-auc:0.931977+0.00137456	test-auc:0.91355+0.0142579
[122]	train-auc:0.93226+0.0013907	test-auc:0.913749+0.0142703
[123]	train-auc:0.932661+0.00139099	test-auc:0.913978+0.0142092
[124]	train-auc:0.932997+0.00127247	test-auc:0.91415+0.0140769
[125]	train-auc:0.933353+0.0013435	test-auc:0.914473+0.0140056
[126]	train-auc:0.933658+0.00132113	test-auc:0.914695+0.0139488
[127]	train-auc:0.934076+0.0013503	test-auc:0.915128+0.0137874
[128]	train-auc:0.934447+0.00142447	test-auc:0.915367+0.0137688
[129]	train-auc:0.93472+0.00145105	test-auc:0.9154

[243]	train-auc:0.954958+0.00102756	test-auc:0.92522+0.0116729
[244]	train-auc:0.95509+0.00103579	test-auc:0.925257+0.0116433
[245]	train-auc:0.955201+0.00106058	test-auc:0.925329+0.0116335
[246]	train-auc:0.955346+0.00109502	test-auc:0.925368+0.0115585
[247]	train-auc:0.955426+0.00109865	test-auc:0.925459+0.0115821
[248]	train-auc:0.955555+0.0010838	test-auc:0.925449+0.0115236
[249]	train-auc:0.955684+0.00105115	test-auc:0.925403+0.0115652
[250]	train-auc:0.955837+0.00101999	test-auc:0.925425+0.0115594
[251]	train-auc:0.955927+0.00102304	test-auc:0.925427+0.0115561
[252]	train-auc:0.95603+0.00101965	test-auc:0.925406+0.0115303
[253]	train-auc:0.956144+0.00101704	test-auc:0.925431+0.0115164
[254]	train-auc:0.956253+0.0010211	test-auc:0.925423+0.011466
[255]	train-auc:0.956338+0.000992652	test-auc:0.925469+0.0114603
[256]	train-auc:0.956496+0.000967417	test-auc:0.925492+0.0114912
[257]	train-auc:0.956578+0.000973772	test-auc:0.925502+0.0113946
[258]	train-auc:0.956688+0.000961195	test-a

[371]	train-auc:0.9675+0.000898983	test-auc:0.92639+0.0107875
[372]	train-auc:0.967608+0.000883112	test-auc:0.926387+0.0107674
[373]	train-auc:0.967668+0.000873218	test-auc:0.926418+0.0107785
[374]	train-auc:0.967773+0.000869026	test-auc:0.926372+0.0107206
[375]	train-auc:0.967852+0.000864393	test-auc:0.926359+0.0106858
[376]	train-auc:0.967933+0.000879775	test-auc:0.926352+0.0106991
[377]	train-auc:0.967998+0.000886856	test-auc:0.926375+0.0106836
[378]	train-auc:0.968078+0.000875387	test-auc:0.926378+0.0107152
[379]	train-auc:0.968171+0.000889864	test-auc:0.926345+0.0107229
[380]	train-auc:0.968243+0.000894812	test-auc:0.92631+0.0107037
[381]	train-auc:0.968325+0.000895511	test-auc:0.926311+0.0106985
[382]	train-auc:0.968436+0.000908324	test-auc:0.926335+0.0106996
[383]	train-auc:0.968522+0.000915693	test-auc:0.926337+0.0106608
[384]	train-auc:0.968603+0.000923935	test-auc:0.926284+0.0106422
[385]	train-auc:0.968687+0.000933944	test-auc:0.926279+0.0106483
[386]	train-auc:0.968748+0.00

In [51]:
test_df['FORTARGET1'] = test_df['FORTARGET1_round0'] * 0.25 + test_df['FORTARGET1_round1'] * 0.25 + test_df['FORTARGET1_round2'] * 0.25 + test_df['FORTARGET1_round3'] * 0.25



In [52]:
print(np.mean(test_df['FORTARGET1']))

def shixin_after_deal(df):
    FORTARGET1 = df['FORTARGET1']
    is_suzhou = df['is_suzhou']
    is_fr = df['is_fr']
    is_fzc = df['is_fzc']
    is_fz = df['is_fz']
    is_qq = df['is_qq']
    is_zp = df['is_zp']
    is_tyshxydm_many = df['is_tyshxydm_many']
    is_zch_many = df['is_zch_many']
    if (is_suzhou == 0) | (is_fr == 1):
        FORTARGET1 = 0
    if is_fzc == 1:
        FORTARGET1 = FORTARGET1 / 1.5
    if is_fz:
        FORTARGET1 = FORTARGET1 / 3
    if is_qq:
        FORTARGET1 = FORTARGET1 / 2
    if is_zp:
        FORTARGET1 = FORTARGET1 / 1.5
    if is_tyshxydm_many > 1:
        FORTARGET1 = 0
    if is_zch_many > 1:
        FORTARGET1 = 0
    return FORTARGET1

test_df['FORTARGET1'] = test_df.apply(shixin_after_deal, axis=1)
print(np.mean(test_df['FORTARGET1']))


0.019024838303959513
0.018173326782235814


In [53]:
chufa_fea = ['zczj', 'fzjgsf', 'fzjgzt', 'fzhydm', 'fzjgqx', 'fzjglx', 'sjly', 'rwbh', 
#               'zckbzj', 
       'QYFZCHRD_yynx', 'sshydm_encoder', 'xzgh', 'qyjd', 'qywd', 
       'QYBZRYXX_number', 'QYFZCHRD_number', 'QYSWDJXX_number', 'FRXZXKZX_number',
       'SDBDJQQJMCF_number', 'SGSFRXZXKXX_number', 'XKZZNJXX_number',
       'ZPSJ_number', 'ZZDJBGXX_number', 'QYJL_number', 
      'zczbbzmc_encoder', 'hymldm_encoder', 'zsszdsf_encoder', 'fzhyml_encoder',
       'xxtgbmmc_encoder', 'zl_encoder', 'sszj_encoder', 'fzjgmc_encoder',
       'qyjglxmc_encoder', 'qylxmc_encoder', 'QYFZCHRD_swglm_encoder',
       'QYFZCHRD_gljg_encoder', 'QYFZCHRD_djzclx_encoder', 
        'SGSFRXZXKXX_dfbm', 'SGSFRXZXKXX_xknr_encoder', 'SGSFRXZXKXX_xmmc_encoder', 'SGSFRXZXKXX_splb_encoder', 
       'SGSFRXZXKXX_xkjg_encoder', 'SGSFRXZXKXX_xysyfw_encoder', 'SGSFRXZXKXX_sjzt2_encoder', 
        'XKZZNJXX_njnd_encoder', 'XKZZNJXX_njjgmc_encoder', 'XKZZNJXX_njsxmc_encoder', 
         'ZZDJBGXX_zzmc_encoder', 'ZZDJBGXX_zyfw_encoder', 'ZZDJBGXX_rdjgqc_encoder', 'ZZDJBGXX_zzzsbh_encoder', 
      'QY_tzrgs', 'QY_zwzls', 'QY_fddbrbz_gs', 'QY_sxdbbz_gs', 
              
        'fzsw_deltaDay', 'clhz_deltaDay', 
       'clrq_month', 'clrq_day', 'hzrq_year', 'hzrq_month', 'hzrq_day', 'clrq_year',
              
#        'ZPSJ_zprs_max', 'ZPSJ_zprs_min', 'ZPSJ_zprs_mean', 'ZPSJ_zprs_std',
              
       'jgdzzs_encoder', 'jyfw_encoder', 'sshymc_encoder',

#                            'jgdzzs_top3_encoder', 'jgdzzs_top6_encoder',
               'QYSWDJXX_fddbrzjmc_encoder', 'QYSWDJXX_shdw_encoder',
 'QYSWDJXX_qy_encoder', 'QYSWDJXX_shjg_encoder', 'QYSWDJXX_djzclx_encoder',
             
#              'QYFZCHRD_zcdz_encoder', 
#     'XKZZNJXX_xxtgbmmc_encoder', 
#      'QYFZCHRD_rwbh_encoder', 'QYFZCHRD_xxtgbmmc_encoder',
#     'SGSFRXZXKXX_xzxkbm_encoder', 'SGSFRXZXKXX_xkjdswh_encoder', 'SGSFRXZXKXX_xxtgbmmc_encoder',
             
#        'ZZDJBGXX_jzsx_deltaDay', 'zzsxcl_deltaDay', 'zzjzcl_deltaDay', 'zzsxhz_deltaDay', 'zzjzhz_deltaDay', 
             
#       'ZPSJ_zylb_kinds', 'ZPSJ_gzjy_kinds',
#        'ZPSJ_gzdd_kinds', 'ZPSJ_zprs_kinds', 'ZPSJ_zwyx_kinds',
#        'ZPSJ_zdxl_kinds', 'ZPSJ_zprq_kinds',
              
#         'SGSFRXZXKXX_xkjdjz_deltaDay',
#       'SGSFRXZXKXX_xkjzq_year', 'SGSFRXZXKXX_xkjzq_month', 'SGSFRXZXKXX_xkjzq_day', 
#       'SGSFRXZXKXX_xkjdrq_year', 'SGSFRXZXKXX_xkjdrq_month', 'SGSFRXZXKXX_xkjdrq_day',  
    
       
#        'clxkjz_deltaDay', 'hzxkjz_deltaDay', 'clxkjd_deltaDay', 'hzxkjd_deltaDay', 
#         'njhz_deltaDay', 'njcl_deltaDay', 
#        'XKZZNJXX_njrq_year', 'XKZZNJXX_njrq_month', 'XKZZNJXX_njrq_day', 
#        'ZZDJBGXX_zzsxq_year', 'ZZDJBGXX_zzsxq_month', 'ZZDJBGXX_zzsxq_day', 
#        'ZZDJBGXX_zzjzq_year', 'ZZDJBGXX_zzjzq_month', 'ZZDJBGXX_zzjzq_day', 
       
#        'czbl_max', 'czbl_min', 'czbl_mean', 'czbl_std',
             
       'fzrq_encoder', 'bghzrq_encoder',
     'gszch_encoder', 'fddbrzjhm_encoder', 'zzjgdm_encoder',
             
             'jyfw_array_0_fea', 'jyfw_array_1_fea', 'jyfw_array_2_fea',
             'jyfw_array_3_fea', 'jyfw_array_4_fea', 'jyfw_array_5_fea',
             'jyfw_array_6_fea', 'jyfw_array_7_fea', 'jyfw_array_8_fea',
             'jyfw_array_9_fea',
       
#        'ZPSJ_wzmc_max', 'ZPSJ_wzmc_min', 'ZPSJ_wzmc_mean',
#  'ZPSJ_wzmc_std', 'ZPSJ_gzjy_max', 'ZPSJ_gzjy_min', 'ZPSJ_gzjy_mean',
#  'ZPSJ_gzjy_std', 'ZPSJ_gzdd_max', 'ZPSJ_gzdd_min', 'ZPSJ_gzdd_mean',
#  'ZPSJ_gzdd_std', 'ZPSJ_zylb_max', 'ZPSJ_zylb_min', 'ZPSJ_zylb_mean',
#  'ZPSJ_zylb_std', 'ZPSJ_zwyx_max', 'ZPSJ_zwyx_min', 'ZPSJ_zwyx_mean',
#  'ZPSJ_zwyx_std', 'ZPSJ_zdxl_max', 'ZPSJ_zdxl_min', 'ZPSJ_zdxl_mean',
#  'ZPSJ_zdxl_std', 'ZPSJ_ywzj_max', 'ZPSJ_ywzj_min', 'ZPSJ_ywzj_mean',
#  'ZPSJ_ywzj_std',
       
#        'SGSFRXZXKXX_is_shixin_sjly_rate', 'SGSFRXZXKXX_is_chufa_sjly_rate',
#  'SGSFRXZXKXX_is_shixin_cjrID_rate', 'SGSFRXZXKXX_is_chufa_cjrID_rate',
#  'SGSFRXZXKXX_is_shixin_xxtgbmbm_rate',
#  'SGSFRXZXKXX_is_chufa_xxtgbmbm_rate', 'SGSFRXZXKXX_is_shixin_splb_rate',
#  'SGSFRXZXKXX_is_chufa_splb_rate', 'SGSFRXZXKXX_is_shixin_dfbm_rate',
#  'SGSFRXZXKXX_is_chufa_dfbm_rate', 'SGSFRXZXKXX_is_shixin_xkjg_rate',
#  'SGSFRXZXKXX_is_chufa_xkjg_rate',
       
      ]




In [54]:
chufa_params = {
        'eta': 0.03,
        'max_depth': 13,
        'gamma': 1,
        'subsample': 0.9,
        'colsample_bytree': 0.95,
        'min_child_weight': 3,
        'max_delta_step': 1,
        'lambda': 30,
#             'nthread': 20,
    }


In [55]:
xgbModel_chufa = XgbModel(feaNames=chufa_fea, params=chufa_params)

# 线下模型
startTime = datetime.now()
xgbModel_chufa.trainCV(train_df[chufa_fea].values, train_df['is_chufa'].values)
xgbModel_chufa.getFeaScore(show=True)
print('training time: ', datetime.now()-startTime)



[0]	train-auc:0.797256+0.00430434	test-auc:0.795352+0.0065977
[1]	train-auc:0.803636+0.00497735	test-auc:0.801544+0.0075344
[2]	train-auc:0.807581+0.00112233	test-auc:0.8053+0.00590046
[3]	train-auc:0.813914+0.00828212	test-auc:0.811114+0.00851175
[4]	train-auc:0.822148+0.0095366	test-auc:0.817965+0.013226
[5]	train-auc:0.827962+0.00782807	test-auc:0.823934+0.0116445
[6]	train-auc:0.833784+0.00319833	test-auc:0.830374+0.00630896
[7]	train-auc:0.835702+0.00336738	test-auc:0.832582+0.0062982
[8]	train-auc:0.838554+0.00345944	test-auc:0.835484+0.00599989
[9]	train-auc:0.83965+0.00302098	test-auc:0.836667+0.00625135
[10]	train-auc:0.840819+0.00298266	test-auc:0.837866+0.00690875
[11]	train-auc:0.841922+0.000973212	test-auc:0.839+0.00474375
[12]	train-auc:0.842594+0.000876752	test-auc:0.839428+0.00481382
[13]	train-auc:0.84389+0.00110023	test-auc:0.840397+0.00479424
[14]	train-auc:0.845601+0.0014517	test-auc:0.84174+0.00577094
[15]	train-auc:0.846648+0.00167708	test-auc:0.842941+0.0048613
[

[128]	train-auc:0.965024+0.000335408	test-auc:0.951708+0.00218484
[129]	train-auc:0.965195+0.000331716	test-auc:0.951792+0.0022158
[130]	train-auc:0.965353+0.000320398	test-auc:0.951904+0.00219668
[131]	train-auc:0.965518+0.000326798	test-auc:0.951995+0.0022043
[132]	train-auc:0.965674+0.000328187	test-auc:0.952095+0.00221856
[133]	train-auc:0.965841+0.000321052	test-auc:0.952212+0.00222441
[134]	train-auc:0.965995+0.000314539	test-auc:0.952308+0.00224934
[135]	train-auc:0.966143+0.000313466	test-auc:0.952393+0.00224697
[136]	train-auc:0.966301+0.000327181	test-auc:0.95249+0.00225135
[137]	train-auc:0.96645+0.000305722	test-auc:0.952604+0.00223683
[138]	train-auc:0.966585+0.000295455	test-auc:0.95271+0.00220951
[139]	train-auc:0.966718+0.000293513	test-auc:0.952804+0.00220543
[140]	train-auc:0.966854+0.000276228	test-auc:0.952884+0.00220312
[141]	train-auc:0.966996+0.000278597	test-auc:0.952973+0.00218944
[142]	train-auc:0.967134+0.000260483	test-auc:0.953064+0.00221494
[143]	train-auc

[253]	train-auc:0.977649+0.000239061	test-auc:0.95744+0.00214715
[254]	train-auc:0.977726+0.000243761	test-auc:0.957449+0.00214521
[255]	train-auc:0.977801+0.000248743	test-auc:0.957459+0.00215154
[256]	train-auc:0.977885+0.000255712	test-auc:0.957484+0.00216169
[257]	train-auc:0.977969+0.000247772	test-auc:0.9575+0.00215505
[258]	train-auc:0.978037+0.000245509	test-auc:0.957515+0.00214777
[259]	train-auc:0.978101+0.000247023	test-auc:0.957521+0.00214802
[260]	train-auc:0.978176+0.000240501	test-auc:0.957537+0.0021536
[261]	train-auc:0.978245+0.000245928	test-auc:0.957544+0.00215646
[262]	train-auc:0.97832+0.000247577	test-auc:0.957551+0.00216877
[263]	train-auc:0.978388+0.000248471	test-auc:0.957567+0.00216609
[264]	train-auc:0.978466+0.000251071	test-auc:0.957563+0.00217647
[265]	train-auc:0.978545+0.000250909	test-auc:0.957583+0.00217844
[266]	train-auc:0.97861+0.000246308	test-auc:0.957593+0.00217894
[267]	train-auc:0.978675+0.000244144	test-auc:0.957598+0.00217659
[268]	train-auc:

[379]	train-auc:0.985263+0.000215719	test-auc:0.958216+0.00226435
[380]	train-auc:0.985303+0.00021585	test-auc:0.95822+0.00226474
[381]	train-auc:0.985349+0.000215189	test-auc:0.958231+0.00226981
[382]	train-auc:0.985395+0.000215024	test-auc:0.958223+0.00227
[383]	train-auc:0.985439+0.000216611	test-auc:0.958216+0.00227163
[384]	train-auc:0.985489+0.000214693	test-auc:0.958222+0.00227382
[385]	train-auc:0.985538+0.000214202	test-auc:0.958227+0.00228311
[386]	train-auc:0.985587+0.000214937	test-auc:0.958223+0.0022886
[387]	train-auc:0.985637+0.000213544	test-auc:0.958235+0.00229598
[388]	train-auc:0.985684+0.000212667	test-auc:0.958231+0.0022914
[389]	train-auc:0.985733+0.000211325	test-auc:0.958237+0.0022914
[390]	train-auc:0.985781+0.000206463	test-auc:0.958238+0.00229289
[391]	train-auc:0.985835+0.000206164	test-auc:0.958242+0.00230018
[392]	train-auc:0.985886+0.000202736	test-auc:0.958248+0.00231014
[393]	train-auc:0.985934+0.000205588	test-auc:0.958248+0.00229983
[394]	train-auc:0.

[504]	train-auc:0.989977+0.000203745	test-auc:0.958224+0.00238848
[505]	train-auc:0.990007+0.000206536	test-auc:0.958221+0.00239092
[506]	train-auc:0.990037+0.00020445	test-auc:0.958222+0.00238411
[507]	train-auc:0.990067+0.000201073	test-auc:0.958225+0.0023862
[508]	train-auc:0.990095+0.000203305	test-auc:0.958231+0.00238497
[509]	train-auc:0.990125+0.000203585	test-auc:0.958216+0.00238935
[510]	train-auc:0.990151+0.000200493	test-auc:0.958213+0.00238984
[511]	train-auc:0.990176+0.00019563	test-auc:0.958217+0.00238367
[512]	train-auc:0.990204+0.000193427	test-auc:0.958223+0.00238271
[513]	train-auc:0.990229+0.000186721	test-auc:0.958223+0.00238606
[514]	train-auc:0.990259+0.000187544	test-auc:0.958224+0.00238925
[515]	train-auc:0.990291+0.000187288	test-auc:0.958217+0.00240349
[516]	train-auc:0.990319+0.000187759	test-auc:0.958218+0.00240954
[517]	train-auc:0.990348+0.000182653	test-auc:0.958214+0.00240452
[518]	train-auc:0.990379+0.000186273	test-auc:0.95822+0.00240529
[519]	train-au

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:106: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [56]:
test_df.loc[:,'FORTARGET2'] = xgbModel_chufa.predict(test_df[chufa_fea].values)
print(np.mean(test_df['FORTARGET2']))


0.20862138271331787


In [57]:
test_df.rename(columns={'qymc':'EID'}, inplace=True)
print(test_df[['EID', 'FORTARGET1', 'FORTARGET2']].head())


                                EID  FORTARGET1  FORTARGET2
0  C06A265AA0BAF3EEF90CB893F22A355C    0.012917    0.032887
1  057B332E88A16EA4BD4C662F9281E708    0.008110    0.355273
2  4B70FC35C51C07048E98C44698BF2123    0.024177    0.248291
3  FFDA9306B3E914BF67ACE0A99BF47DA6    0.021981    0.014348
4  CF94736A17FE3078D62C5C2BBEF01033    0.000223    0.014174


In [58]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../temp/%s.csv' % fileName, header=True, index=False)
    
exportResult(test_df[['EID', 'FORTARGET1', 'FORTARGET2']], 'xgb_online')
